In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

In [2]:
def advec_diffuse(a,dt,T,x=[],u0=[]):
    """Uses a spectral decomposition and forward Euler to solve
    the 1D advection diffusion equation u_t + au_x = u_xx.
    Assumes the domain is [0,2pi] or else the wavenumbers would
    need to be weighted differently. a is the advection speed
    dt is the time-step T is the final time we run to x is the
    spatial discretization u0 is the initial condition Because
    this uses the fft, it is best to have len(x) = 2^K - for
    some integer K
    """
    N = len(x)
    #The most difficult part is figuring out how numpy indexes the wavenumbers
    k = np.array([1j*y for y in range(0,int(N/2))] + [0] + [1j*y for y in range(int(-N/2)+1,0)])
    k2 = k**2;
    print(dt*np.max(np.abs(-a*k + k2))-1)
    u = []
    u.append(u0)
    uhat = np.fft.fft(u0)
    totalStep = int(T/dt)
    for nn in range(totalStep):
        # forward Euler time-stepping
        new_uhat = uhat + dt*(-a*k + k2)*uhat
        uhat = new_uhat.copy()
        # revert back to real values (not necessary unless we want
        # to save our solution in real space)
        u.append(np.real(np.fft.ifft(uhat)))
    return x, np.array(u), dt

def test_vals():
    N = 128
    x = np.linspace( 0, 2*np.pi, N )
    a = 4
    T= 0.5
    dt = 1/4000
    u0 = np.sin( x**2 ) + np.cos( x*5 )
    
    return advec_diffuse( a, dt, T, x, u0)

#get values
x, u, dt = test_vals()

#creae animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(150)
ax = fig.add_subplot(111)
ax.set_xlim(0, 2*np.pi)
plt.suptitle(r'Advection Diffusion Equation $\Delta t = $' + str(dt))
#update function
def update(i):
    ax.clear()
    ax.set_ylim(-2.1, 2.1)
    ax.plot(x, u[i])
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    return ax

ani = animation.FuncAnimation(fig, update,
                              frames = range(u.shape[0]), 
                              interval=10)
HTML(ani.to_html5_video())

-0.005752011568542326


# Exercise 10.15
Recall that the trapezoidal method is defined by

\begin{align*}
u(t + \Delta t) \approx u(t) + \frac{\Delta t}{2}\left[f(u(t)) + f(u(t + \Delta t)) \right]
\end{align*}

but remember our ode we are trying to solve is $u' = \lambda u$ meaning $f(u(t)) = \lambda u(t)$.
Even though the trapezoidal method is an implicit method we can still solve explicitly for $u(t + \Delta t)$ because our
vector field is linear.

\begin{align*}
u(t + \Delta t) &\approx u(t) + \frac{\Delta t}{2}\left[\lambda u(t) + \lambda u(t + \Delta t) \right]\\
u(t + \Delta t) \left[1  - \frac{\Delta t \lambda}{2}\right] & \approx u(t)\left[1 + \frac{\Delta t \lambda}{2}\right]\\
u(t + \Delta t) &\approx \frac{u(t)\left[1 + \frac{\Delta t \lambda}{2} \right]}{1 - \frac{\Delta t \lambda}{2} }\\
u(t + \Delta t) &\approx u(t) + \left[\frac{2 + \Delta t \lambda}{2 - \Delta t \lambda} \right]
\end{align*}

Remember that $\lambda_k = - aik - k^2$ where $k$ is the wave number.
This leads to the following algorithm:

In [7]:
def advec_diffuse_trap(a,dt,T,x=[],u0=[]):
    """Uses a spectral decomposition and forward Euler to solve
        the 1D advection diffusion equation u_t + au_x = u_xx.
        Assumes the domain is [0,2pi] or else the wavenumbers would
        need to be weighted differently.
        a is the advection speed
        dt is the time-step
        T is the final time we run to
        x is the spatial discretization
        u0 is the initial condition
        Because this uses the fft, it is best to have len(x) = 2^K
        or some integer K
    """

    N = len(x)
    #The most difficult part is
    #figuring out how numpy indexes the wavenumbers
    k = np.array([1j*y for y in range(0,int(N/2))] + [0] + [1j*y for y in range(int(-N/2)+1,0)])
    k2 = k**2;
    u = []
    u.append(u0)
    uhat = np.fft.fft(u0)
    totalStep = int(T/dt)
    for nn in range(totalStep):
        # trapezoid time-stepping
        new_uhat = uhat * (2 + dt *(-a*k + k2)) / (2 - dt*(-a*k + k2))
        uhat = new_uhat.copy()
        # revert back to real values (not necessary unless we want
        # to save our solution in real space)
        u.append(np.real(np.fft.ifft(uhat)))
    return x, np.array(u), dt

def problem10_15_vals():
    N = 128
    x = np.linspace( 0, 2*np.pi, N )
    a = 4
    T= 0.5
    dt = 1/250
    u0 = np.sin( x**2 ) + np.cos( 5 * x )
    
    return advec_diffuse_trap( a, dt, T, x, u0)

#get values
x, u, dt = problem10_15_vals()
#creae animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(150)
ax = fig.add_subplot(111)
ax.set_xlim(0, 2*np.pi)
plt.suptitle(r'Advection Diffusion Equation $\Delta t = $' + str(dt))
#update function
def update(i):
    ax.clear()
    ax.set_ylim(-2.1, 2.1)
    ax.plot(x, u[i])
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    return ax

ani = animation.FuncAnimation(fig, update,
                              frames = range(u.shape[0]), 
                              interval=35)
HTML(ani.to_html5_video())

# Exercise 10.16

We now derive the forward Euler discretization of the Kuramato-Sivashinsky (KS) equation on $[0, 2\pi]$:
\begin{align*}
u_t + uu_{x} + u_{xx} + u_{xxxx} &=0
\end{align*}
Notice that this is a quasi-linear PDE so we will use eigenfunction expansions for quasi-linear PDEs and a numerical pseudo-spectral method. We first consider the ansatz $u(x, t) = \sum_{k} \widehat{u}_k(t) e^{ikx}$
plugging this into our PDE we get

\begin{align*}
\sum_{k} \widehat{u}_k'(t) e^{ikx} + \sum_{k} \widehat{u}_k(t)e^{ikx}\sum_{n} i n \widehat{u}_n(t) e^{inx} -k^2 \sum_{k}\widehat{u}_k(t) e^{ikx} + k^4 \sum_{k}k^4 \widehat{u}_k(t) e^{ikx} &=0
\end{align*}

Recall that $e^{ikx}$ is an orthogonal basis of $L^2(\Omega)$ (we proved this in volume 2) using this we get

\begin{align*}
\sum_{k} \widehat{u}_k'(t) \int_0^{2\pi} e^{ikx} e^{imx} + \sum_{k, n} \widehat{u}_k(t) \cdot in \widehat{u}_n(t) \int_0^{2\pi} e^{ikx} e^{inx} e^{imx} dx -k^2 \sum_{k}\widehat{u}_k(t)\int_0^{2\pi} e^{ikx} e^{imx} dx + k^4 \sum_{k}k^4 \widehat{u}_k(t) \int_0^{2\pi} e^{ikx}e^{imx}dx &=0
\end{align*}

And notice that

\begin{align*}
    \int_0^{2\pi} e^{ikx} e^{inx} e^{imx} dx = \frac{-i\left(-1 + e^{2i\pi (k +m +n)} \right)}{k+m+n}= \frac{-i\left(-1 + \cos(2\pi(k+m+n)) +i \sin(2\pi (k+m+n)) \right)}{k+m+n} = 0 \textrm{ if } n\neq m\neq k
\end{align*}

Which allows us to give our formula as

\begin{align*}
    \widehat{u}_t + \widehat{u}\cdot ik \widehat{u} - k^2 \widehat{u} + k^4 \widehat{u} &= 0\\
    \widehat{u}_t + \mathscr{F}\left( \mathscr{F}^{-1}(\widehat{u}) \cdot \mathscr{F}^{-1}(ik\widehat{u})\right) + (k^4 - k^2)\widehat{u} &=0
\end{align*}

Inserting our forward Euler method leads to 

\begin{align*}
\frac{\widehat{u}(t + \Delta t) - \widehat{u}(t))}{\Delta t} + \mathscr{F}\left( \mathscr{F}^{-1}(\widehat{u}(t)) \cdot \mathscr{F}^{-1}(ik\widehat{u}(t))\right) + (k^4 - k^2)\widehat{u}(t) = 0\\
\widehat{u}(t + \Delta t) = \widehat{u}(t) + \Delta t\left[ (k^2 - k^4) \widehat{u}(t) -\mathscr{F}\left( \mathscr{F}^{-1}(\widehat{u}(t)) \cdot \mathscr{F}^{-1}(ik\widehat{u}(t))\right) \right]
\end{align*}

which is our update function. This is expressed in the following algorithm:

In [11]:
def diffusive_burgers(dt,T,x=[],u0=[]):
    """Uses a spectral decomposition and forward Euler to solve
        the 1D diffusive Burgers equation u_t + uu_x = u_xx. Assumes
        the domain is [0,2pi] or else the wavenumbers would need to
        be weighted differently. 
        dt is the time-step 
        T is the final time we run to 
        x is the spatial discretization u0 is the
        initial condition Because this uses the fft, 
        it is best to have len(x) = 2^K - for some integer K
    """
    N = len(x)
    #The most difficult part is figuring out how numpy indexes the wavenumbers
    k = np.array([y*1j for y in range(0,int(N/2))] + [0] + [y*1j for y in range(int(-N/2)+1,0)])
    k2 = k**2;
    k4 = k**4
    u = []
    u.append(u0)
    uhat = np.fft.fft(u0)
    totalStep = int(T/dt)
    for nn in range(totalStep):
        # forward Euler time-stepping
        new_uhat = uhat - dt*(np.fft.fft(np.fft.ifft(k*uhat)*np.fft.ifft(uhat)) + (k2 + k4)*uhat )
        uhat = new_uhat.copy()
        # revert back to real values (not necessary unless we want
        # to save our solution in real space)
        u.append(np.real(np.fft.ifft(uhat)))
    return x,np.array(u), dt

# Exercise 10.17

In [13]:
def problem10_17_vals():
    N = 128
    x = np.linspace( 0, 2*np.pi, N )
    T= 0.5
    dt = 1e-6
    u0 = np.sin( x**2 ) + np.cos( 5 * x )
    
    return diffusive_burgers( dt, T, x, u0)

#get values
x, u, dt = problem10_17_vals()
print(u.shape)
#creae animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(150)
ax = fig.add_subplot(111)
ax.set_xlim(0, 2*np.pi)
plt.suptitle(r' Kuramato-Sivashinsky (KS) equation $\Delta t = $' + str(dt))
#update function
def update(i):
    ax.clear()
    ax.set_ylim(-2.1, 2.1)
    ax.plot(x, u[i])
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    return ax

ani = animation.FuncAnimation(fig, update,
                              frames = range(500), 
                              interval=60)
HTML(ani.to_html5_video())

(500001, 128)


In [3]:
def KS_forward(dt, T, x=[], u0=[]):
    L = np.max(x)
    nu = 1
    nx = len(x)

    t0 = 0 
    tN = T
    nt = int(T/dt)
    
    # wave number mesh
    k = np.arange(-nx/2, nx/2, 1)

    t = np.linspace(start=t0, stop=tN, num=nt)
    x = np.linspace(start=0, stop=L, num=nx)

    # solution mesh in real space
    u = np.ones((nx, nt))
    # solution mesh in Fourier space
    u_hat = np.ones((nx, nt), dtype=complex)

    u_hat2 = np.ones((nx, nt), dtype=complex)

    # initial condition 
    #u0 = np.cos((2 * np.pi * x) / L) + 0.1 * np.cos((4 * np.pi * x) / L)

    # Fourier transform of initial condition
    u0_hat = (1 / nx) * np.fft.fftshift(np.fft.fft(u0))

    u0_hat2 = (1 / nx) * np.fft.fftshift(np.fft.fft(u0**2))

    # set initial condition in real and Fourier mesh
    u[:,0] = u0
    u_hat[:,0] = u0_hat

    u_hat2[:,0] = u0_hat2

    # Fourier Transform of the linear operator
    FL = (((2 * np.pi) / L) * k) ** 2 - nu * (((2 * np.pi) / L) * k) ** 4
    # Fourier Transform of the non-linear operator
    FN = - (1 / 2) * ((1j) * ((2 * np.pi) / L) * k)

    # resolve EDP in Fourier space
    for j in range(0,nt-1):
        uhat_current = u_hat[:,j]
        uhat_current2 = u_hat2[:,j]
        if j == 0:
            uhat_last = u_hat[:,0]
            uhat_last2 = u_hat2[:,0]
        else:
            uhat_last = u_hat[:,j-1]
            uhat_last2 = u_hat2[:,j-1]
  
        # compute solution in Fourier space through a finite difference method
        # Cranck-Nicholson + Adam 
        u_hat[:,j+1] = (1 / (1 - (dt / 2) * FL)) * ( (1 + (dt / 2) * FL) * uhat_current + ( ((3 / 2) * FN) * (uhat_current2) - ((1 / 2) * FN) * (uhat_last2) ) * dt )
        # go back in real space
        u[:,j+1] = np.real(nx * np.fft.ifft(np.fft.ifftshift(u_hat[:,j+1])))
        u_hat2[:,j+1] = (1 / nx) * np.fft.fftshift(np.fft.fft(u[:,j+1]**2))
        
    return x, u.T, dt


def problem10_18_vals():
    N = 128
    x = np.linspace( 0, 2*np.pi, N )
    T= 0.5
    dt = 1/7000
    u0 = np.sin( x**2 ) + np.cos( 5 * x )
    
    return KS_forward( dt, T, x, u0)
#get values
x, u, dt = problem10_18_vals()
#create animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(150)
ax = fig.add_subplot(111)
plt.suptitle(r' Kuramato-Sivashinsky (KS) equation $\Delta t = $' + str(dt))
#update function
def update(i):
    ax.clear()
    ax.set_ylim(-2.1, 2.1)
    ax.set_xlim(0.25, 2*np.pi-0.25)
    ax.plot(x, u[i])
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    return ax

ani = animation.FuncAnimation(fig, update,
                              frames = range(1200), 
                              interval=60)
HTML(ani.to_html5_video())

here


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

xx, tt = np.meshgrid(x, t)
levels = np.arange(-3, 3, 0.01)
cs = ax.contourf(xx, tt, u.T, cmap=cm.jet)
fig.colorbar(cs)

ax.set_xlabel("x")
ax.set_ylabel("t")
ax.set_title(f"Kuramoto-Sivashinsky: L = {L}, nu = {nu}")